In [ ]:
!pip install netron

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import netron
from tensorflow.keras.utils import plot_model

In [ ]:
def split_zo_conv(ll, ZO_cut):
    "Split a convolution layer ll into pieces in Z as given by ZO_cut. The filters of layer equal sum(ZO_cut)"

    # Get  configuration
    cfg = ll.get_config()

    # Check we are dealing with simple case
    if not( cfg['dilation_rate']==(1,1) or cfg['data_format']=='channels_last' or cfg['activation']=='linear'):
            raise Exception("Sorry, this case for convolution not implemented yet")

    # Extract weights and bias
    if cfg['use_bias']:
        w, bias = ll.get_weights()
    else:
        w,      = ll.get_weights()

    name_t = cfg['name'] + "_"
    cfg_t = cfg.copy()
    del cfg_t['filters']
    del cfg_t['name']

    # Create the new conv layers
    lls = [layers.Conv2D(filters=flt, name=name_t + str(it), **cfg_t) for it, flt in enumerate(ZO_cut)]

    # Create a tensor Input
    x = layers.Input(shape=ll.input_shape[1:]) # Note that BatchSize=None is removed..

    # Connect tensor input to layers
    lls_tf = [lli(x) for lli in lls]

    # Add weights
    idx=0
    for lli, flt in zip(lls, ZO_cut):
        if cfg['use_bias']:
            lli.set_weights([w[:,:,:, idx:idx+flt].copy(), bias.copy()])
        else:
            lli.set_weights([w[:,:,:, idx:idx+flt].copy()])
        idx += flt

    # Concatenate outputs
    y = layers.Concatenate()(lls_tf)  # Concatenate in axis=-1

    m = models.Model(x, y)

    return m

In [ ]:
def split_yo_conv(ll, YO_cut):
    """It splits a convolution layer =ll= into pieces in Y direction
    as given by the list YO_cut. The number of lines of layer equal sum(YO_cut)

    It assumes no dilation, standard data_format, and
    """

    # Get  configuration
    cfg = ll.get_config()

    # Check we are dealing with simple case
    if not( cfg['dilation_rate']==(1,1) or cfg['data_format']=='channels_last' or cfg['activation']=='linear'):
            raise Exception("Sorry, this case for convolution not implemented yet")

    # Limits of YO
    Ky = cfg['kernel_size'][1]  # Kernel size in Y direction
    Sy = cfg['strides'][1]      # Stride in Y direction
    lim = np.cumsum([0] + YO_cut)
    # Regions in input (accounting for stride and kernel size)
    reg = list([Sy*r, Sy*(s-1) + Ky-1 ] for r, s in   zip(lim[:-1], lim[1:]))

    # Create the new conv layers with same params as initial one, but name
    name_t = cfg['name'] + "_"
    cfg_t = cfg.copy()
    del cfg_t['name']
    lls = [layers.Conv2D(name=name_t + str(it), **cfg_t) for it,_ in enumerate(YO_cut) ]

    # Create a tensor Input of same size as initial
    x = layers.Input(shape=ll.input_shape[1:]) # Note that BatchSize=None is removed..

    # Connect tensor input to layers according to region limits
    lls_tf = [lli(x[:,:,r:s+1,:]) for (r, s), lli in (zip(reg, lls))]  # Note +1 because of python limits

    # Add weights
    for lli in (lls):
        lli.set_weights(ll.get_weights())  # Note it is a link, not a copy!

    # Concatenate outputs
    y = layers.Concatenate(axis=-2)(lls_tf)  # Concatenate in axis=-2 (ie., Y)

    m = models.Model(x, y)

    return m


In [ ]:
# Read data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

# Create Model
model = models.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

# Plotting the model
# Using netron
# model_name = "Original model.h5"
# model.save(model_name)
# netron.start(model_name,8088)

# Using utils
plot_model(model)

# Train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=2, batch_size=128, validation_split=0.1 )
score = model.evaluate(x_test, y_test, verbose=0)



# Now create a new model removing a conv layer by the one we have generated

ll = model.layers[3]

ZO_cut = [32, 16, 16]

aux = split_zo_conv(ll, ZO_cut)

modelz_lst = model.layers
modelz_lst[3] = aux

modelz = models.Sequential(modelz_lst)


modelz.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Plotting the modified model
# Using netron
model_name = "Modified model.h5"
modelz.save(model_name)
netron.start(model_name,8089)
plot_model(modelz)
score2 = modelz.evaluate(x_test, y_test, verbose=0)

print(score)
print(score2)

Epoch 1/2
422/422 [==============================] - 25s 56ms/step - loss: 0.5454 - accuracy: 0.8094 - val_loss: 0.7105 - val_accuracy: 0.7512
Epoch 2/2
422/422 [==============================] - 23s 55ms/step - loss: 0.3762 - accuracy: 0.8654 - val_loss: 0.3206 - val_accuracy: 0.8815
Serving 'Modified model.h5' at http://localhost:8089
[0.3383619487285614, 0.8780999779701233]
[0.3383619487285614, 0.0997999981045723]


In [ ]:
# Read data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

# Create Model
model = models.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="valid",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

# Train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=1, batch_size=128, validation_split=0.1 )
score = model.evaluate(x_test, y_test, verbose=0)



# Now create a new model removing a conv layer by the one we have generated

ll = model.layers[3]
print(ll.output_shape)
YO_cut = [3, 3]

aux = split_yo_conv(ll, YO_cut)

model2_lst = model.layers
model2_lst[3] = aux

model2 = models.Sequential(model2_lst)
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


score2 = model2.evaluate(x_test, y_test, verbose=0)

print(score)
print(score2)

422/422 [==============================] - 21s 47ms/step - loss: 0.5507 - accuracy: 0.8039 - val_loss: 0.6734 - val_accuracy: 0.7952
(None, 6, 6, 64)
[0.6847608685493469, 0.791700005531311]
[0.6847608685493469, 0.791700005531311]
